In [ ]:
import tifffile
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import colorcet as cc

%matplotlib widget

input_folder = Path(r"/Users/simonvanstaalduine/AMOLF-SHIMIZU Dropbox/DATA/NETWORK_VISU/052_SAL2506A/stitch_stabilized")
input_files = sorted(list(input_folder.glob("*.tif")))
output_folder = Path(r"/Users/simonvanstaalduine/AMOLF-SHIMIZU Dropbox/DATA/NETWORK_VISU/052_SAL2506A/stitch_difference")

In [ ]:
im_index = 122
for im_index in range(0, len(input_files) - 1):
    test_im_a = tifffile.imread(input_files[im_index])
    test_im_b = tifffile.imread(input_files[im_index + 1])

    # Invert images
    test_im_a = np.invert(test_im_a)
    test_im_b = np.invert(test_im_b)

    difference = test_im_b - test_im_a
    difference = np.where(difference < 15, 0, difference)  # Ensure no negative values
    difference = np.where(difference > 150, 0, difference)  # Ensure no values above 255

    tifffile.imwrite(output_folder / f"diff_{im_index:04d}.tif", difference.astype(np.uint8))

# fig, axs = plt.subplots(4, figsize=(10, 20))
# ensure axs contains axis type

# axs[0].imshow(test_im_a, cmap='gray')
# axs[0].set_title("Image A")
# axs[1].imshow(test_im_b, cmap='gray')
# axs[1].set_title("Image B")
# axs[2].imshow(difference, cmap='viridis')
# axs[3].hist(difference.ravel(), bins=100, color='blue', alpha=0.7, log=True)


In [ ]:
import numpy as np
import tifffile
import matplotlib.pyplot as plt
from matplotlib import cm
import datetime

# === PARAMETERS ===
input_tiff_path = '/Users/simonvanstaalduine/AMOLF-SHIMIZU Dropbox/DATA/NETWORK_VISU/118_ART2504A/stitch_difference_2'
output_tiff_path = 'colored_projection.tiff'
colormap_name = 'rainbow4'     # or 'viridis', 'inferno', etc.
threshold = .01      # adjust to mask out background

input_files = sorted(list(Path(input_tiff_path).glob("*.tif")))

# === 1. Load the TIFF stack ===
# This loads a 3D numpy array: (frames, height, width)
stack = tifffile.imread(input_files)
num_frames, height, width = stack.shape

print(f"Loaded stack with {num_frames} frames of size {height}x{width}")

# === 2. Normalize frames ===
# Normalize intensity to 0–1 for each frame
stack = stack.astype(np.float32)
stack -= stack.min()
stack /= stack.max()

# === 3. Set up colormap ===
cmap = cc.cm[colormap_name]
norm_times = np.linspace(0, 1, num_frames)

# === 4. Initialize RGB output ===
rgb_image = np.zeros((height, width, 3), dtype=np.float32)

# === 5. Build the color-projection image ===
for i in range(num_frames-1, 0, -1):
    frame = stack[i]
    mask = frame > threshold   # Only keep bright particles
    frame = np.where(frame > .0001, .0001, frame)
    color = np.array(cmap(norm_times[i])[:3])  # RGB only, ignore alpha
    rgb_layer = mask[..., None]  * color # * frame[..., None]
    # reassign pixels
    rgb_image[mask] = rgb_layer[mask]
    # rgb_image += rgb_layer

# === 6. Normalize the result ===
rgb_image /= rgb_image.max()

# === 7. Convert to 8-bit for saving ===
rgb_image_8bit = (rgb_image * 255).astype(np.uint8)

im_ratio = height / width

# Make a plt plot out of it with colorbar
fig, ax = plt.subplots(figsize=(10, 7))
img_show = ax.imshow(rgb_image_8bit, cmap=cmap)
ax.axis('off')
ax.set_title(f"Color-encoded time projection plate ART118")
# plt.colorbar(label='Time of growth')

# set colorbar ticks in hours using a datetime interval
total_hours = num_frames * 2
ticks = np.linspace(0, total_hours, 5)
tick_labels = [str(datetime.timedelta(hours=int(tick))) for tick in ticks]
fig.colorbar(img_show, ticks=ticks, label='Time of growth (hours)', fraction=0.046 * im_ratio, pad=0.04)
# adjust colorbar size to fit the figure

fig.tight_layout()

# === 8. Save as TIFF ===
tifffile.imwrite(output_tiff_path, rgb_image_8bit)

print(f"Saved color-encoded projection to {output_tiff_path}")
